In [1]:
!pip install pyTelegramBotAPI
!pip install category_encoders
!pip install catboost
!pip install lightgbm
!pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.2 MB/s eta 0:00:00


In [2]:
import telebot

In [3]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV
from category_encoders import OrdinalEncoder, OneHotEncoder, CountEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from imblearn.over_sampling import SMOTE

from google.colab import files
from tqdm.notebook import tqdm
import os
import zipfile
import json
import pickle

In [4]:
TOKEN = "*"

In [6]:
def min_max_norm(x, col):
    return (x - col.min()) / (col.max() - col.min())

In [7]:
dataset = files.upload()

Saving finish_dataset.csv to finish_dataset.csv


In [9]:
df = pd.read_csv('finish_dataset.csv')

In [ ]:
df.head()

,absolute_magnitude,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous,velocity_magnitude_ratio
0,19.14,0.883161,71745.401048,5.814362e+07,False,0.614111
1,18.50,1.185878,109949.757148,5.580105e+07,True,1.004114
2,21.45,0.304818,24865.506798,6.720689e+07,False,0.172295
3,20.63,0.444672,78890.076805,3.039644e+07,False,0.582245
4,22.70,0.171412,56036.519484,6.311863e+07,False,0.348412


In [10]:
model = files.upload()

Saving classifier_model_v1.pkl to classifier_model_v1.pkl


In [11]:
with open('classifier_model_v1.pkl', 'rb') as file:
    classifier = pickle.load(file)

In [19]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['pred'])
def handle_pred_command(message):
    try:
        args = message.text[len('/pred'):].strip()

        numbers = eval(args)

        if isinstance(numbers, list):
            if len(numbers) < 4:
                bot.reply_to(message, "Error: Please enter 4 numbers.")
            else:
                abs_m = numbers[0]
                diam = numbers[1]
                vel = numbers[2]
                dist = numbers[3]
                vmr = min_max_norm(vel, df["relative_velocity"])/min_max_norm(abs_m, df["absolute_magnitude"])

                pred = classifier.predict([[abs_m, diam, vel, dist, vmr]])

                if pred[0]:
                    response = f"NEO({numbers[:4]}) is hazardous!"
                else:
                    response = f"NEO({numbers[:4]}) is not hazardous."

                bot.reply_to(message, response)
        else:
            bot.reply_to(message, "Error: Please enter a list of numbers.")
    except Exception as e:
        bot.reply_to(message, f"Error: {str(e)}")

@bot.message_handler(commands=['help'])
def handle_help_command(message):

    text = '''
    List of commands:
    /help - list of commands
    /pred [<abs_mag>, <diam>, <vel>, <dist>] - predict the danger of N.E.O.
        <abs_mag> - describes intrinsic luminosity
        <diam> - maximum estimated diameter in kilometres
        <vel> - velocity relative to Earth in kmph
        <dist> - distance in kilometres missed
    '''

    bot.reply_to(message, text)

bot.polling()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
